Jan 2021

Takes a Planet image and applies the accompanying UDM2 mask to mask 
clouds and shadows for an entire order.  

Writes masked bands to a new .tif file.

***Windows users: You will need to change the direction of the slashes in the filepaths***

Resources used to develop this notebook: 
- https://github.com/planetlabs/notebooks/blob/master/jupyter-notebooks/udm/udm.ipynb
- https://notebook.community/planetlabs/notebooks/jupyter-notebooks/udm2/udm2

In [1]:
import os
import rasterio
import numpy as np

In [24]:
# To mask a whole order:

# If NOT clipped: Set to folder ending with 'PSScene4Band'.  
# If clipped, set to folder 'files'
main_dir = '.../PSScene4Band'

# If you used the Planet API or Explorer to clip images, set to "True"  It
# affects the file names and way data is delivered.
clipped_to_AOI = False

# Older images may not have a UDM2.  Set to 'False' if this is the case.
udm2_available = True

# Depending on needs, you may not want to use the UDM mask.  It can take out
# a lot of pixels sometimes.
use_udm = True

In [25]:
if clipped_to_AOI == True:
    os.chdir(main_dir)
    filenames = []
    
    files = os.listdir()
    
    for name in files:
        if name.endswith("R_clip.tif"):
            file = name[:-23]
            filenames.append(file)
        
else:
    # Change directory to location of files with individual images:
    os.chdir(main_dir)

    # Get a list of image names:
    files = os.listdir()
    filenames = []
    for file in files:
        file = file + '_3B'
        filenames.append(file)

    new_wrk_dirs = []
    for file in files:
        new_dir = main_dir + '/' + file + '/analytic_sr_udm2'
        new_wrk_dirs.append(new_dir)

In [27]:
def load_udm(udm_filename):
    '''Load single-band bit-encoded UDM as a 2D array.'''
    with rasterio.open(udm_filename, 'r') as src:
        udm = src.read()
    return udm

def udm_to_mask_all(udm):
    '''Create a mask from the udm, masking all pixels with quality concerns''' 
    return udm != 0

def load_udm2(udm2_filename):
    '''Load multi-band UDM2 as a 3d array.'''
    with rasterio.open(udm2_filename, 'r') as src:
        udm2 = src.read()
    return udm2

def mask_cloud_shadow(udm2_array):
    '''Create a mask from the udm2, masking all cloud and cloud shadow pixels'''
    shadow_band = udm2_array[2,...]
    cloud_band = udm2_array[5,...]
    masked_pixels = np.logical_or(shadow_band == 1, cloud_band == 1)
    return masked_pixels

In [28]:
if clipped_to_AOI == True:
    for filename in filenames:
        udm_filename = filename +'_AnalyticMS_DN_udm_clip.tif'
        udm2_filename = filename + '_udm2_clip.tif'
        image_filename = filename + '_AnalyticMS_SR_clip.tif'
        masked_image_filepath = filename + '_AnalyticMS_SR_clip_masked.tif'

        udm = load_udm(udm_filename)    
        udm_mask = udm_to_mask_all(udm)

        if udm2_available == True:    
            udm2 = load_udm2(udm2_filename)
            udm2_mask = mask_cloud_shadow(udm2)

        with rasterio.open(image_filename, 'r') as src1:
                org_image = src1.read()

        band_cnt = range(0, src1.count)

        masked_band_list = []

        for b in band_cnt:
            band = org_image[b]

            arr = np.zeros((np.shape(udm_mask)[0], np.shape(udm_mask)[1]),  dtype=np.uint16)

            if udm2_available == True:
                tmp = np.ma.masked_array(band, udm2_mask)
                if use_udm == True:
                    tmp = np.ma.masked_array(tmp, udm_mask) 
            else:
                tmp = np.ma.masked_array(band, udm_mask)

            arr = np.ma.filled(tmp, 0)

            masked_band_list.append(arr)

        masked_band_array = np.array(masked_band_list)

        src1 = rasterio.open(image_filename, 'r')

        with rasterio.Env():
            profile=src1.profile
            profile.update({'nodata': 0})
            with rasterio.open(masked_image_filepath, 'w',
                               **profile) as dst:
                dst.write(masked_band_array)
                
else:
    for num, folder in enumerate(new_wrk_dirs):
        os.chdir(folder)
        filename = filenames[num]

        udm_filename = filename +'_AnalyticMS_DN_udm.tif'
        udm2_filename = filename + '_udm2.tif'
        image_filename = filename + '_AnalyticMS_SR.tif'
        masked_image_filepath = filename + '_AnalyticMS_SR_masked.tif'

        udm = load_udm(udm_filename)    
        udm_mask = udm_to_mask_all(udm)

        if udm2_available == True:    
            udm2 = load_udm2(udm2_filename)
            udm2_mask = mask_cloud_shadow(udm2)

        with rasterio.open(image_filename, 'r') as src1:
                org_image = src1.read()

        band_cnt = range(0, src1.count)

        masked_band_list = []

        for b in band_cnt:
            band = org_image[b]

            arr = np.zeros((np.shape(udm_mask)[0], np.shape(udm_mask)[1]),  dtype=np.uint16)

            if udm2_available == True:
                tmp = np.ma.masked_array(band, udm2_mask)
                if use_udm == True:
                    tmp = np.ma.masked_array(tmp, udm_mask) 
            else:
                tmp = np.ma.masked_array(band, udm_mask)

            arr = np.ma.filled(tmp, 0)

            masked_band_list.append(arr)

        masked_band_array = np.array(masked_band_list)

        src1 = rasterio.open(image_filename, 'r')

        with rasterio.Env():
            profile=src1.profile
            profile.update({'nodata': 0})
            with rasterio.open(masked_image_filepath, 'w',
                               **profile) as dst:
                dst.write(masked_band_array)